In [51]:
from datasets import load_dataset
import os
import tarfile
from pathlib import Path
import glob
from itertools import chain
import polars as pl
import numpy as np
!uv pip install bs4

Using Python 3.13.11 environment at: /Users/sahila/Documents/Self Learning/transformers_hf/.venv
Audited 1 package in 1ms


In [27]:
BASE_DIR = Path(os.getcwd())

In [5]:
dataset = load_dataset("fka/awesome-chatgpt-prompts")
dataset

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt', 'for_devs', 'type', 'contributor'],
        num_rows: 810
    })
})

In [8]:
dataset['train'][0]

{'act': 'Ethereum Developer',
 'prompt': 'Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.',
 'for_devs': True,
 'type': 'TEXT',
 'contributor': 'ameya-2003'}

In [13]:
dataset['train'].shuffle(seed= 37).select(range(100))

Dataset({
    features: ['act', 'prompt', 'for_devs', 'type', 'contributor'],
    num_rows: 100
})

In [21]:
## split into train, test
dataset['train'].train_test_split(
    train_size = 0.8, seed = 42
)

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt', 'for_devs', 'type', 'contributor'],
        num_rows: 648
    })
    test: Dataset({
        features: ['act', 'prompt', 'for_devs', 'type', 'contributor'],
        num_rows: 162
    })
})

### Creating your own dataset using a sgm file

In [22]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz

--2025-12-30 09:55:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘reuters21578.tar.gz’

reuters21578.tar.gz     [  <=>               ]   7.77M  21.7MB/s    in 0.4s    

2025-12-30 09:55:56 (21.7 MB/s) - ‘reuters21578.tar.gz’ saved [8150596]



In [29]:
import tarfile
with tarfile.open(BASE_DIR/"my_model"/"datasets"/"reuters21578.tar.gz", 'r:gz') as tar:
    tar.extractall(path=BASE_DIR/"my_model"/"datasets"/"destination_folder")

/var/folders/pz/5t94392x1t1_xpdnb9_w0_0r0000gn/T/ipykernel_41571/4224340313.py:3: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=BASE_DIR/"my_model"/"datasets"/"destination_folder")


In [47]:
from bs4 import BeautifulSoup

def parse_sgm(file_path):
    with open(file_path, 'r', encoding='latin-1') as f:
        content = f.read()

    soup = BeautifulSoup(content, 'html.parser')

    documents = []
    for reuters in soup.find_all('reuters'):
        doc = {
            'title': reuters.title.get_text() if reuters.title else '',
            'body': reuters.body.get_text() if reuters.body else '',
            'topics': [topic.get_text() for topic in reuters.find_all('d')]
        }
        documents.append(doc)

    return documents

all_sgm_files = glob.glob(str(BASE_DIR/"my_model"/"datasets"/"destination_folder")+"/*.sgm")
articles = []
for each_file in all_sgm_files:
    articles.append(parse_sgm(each_file))
articles = list(chain.from_iterable(articles))

## load a parquet file

In [92]:
np.random.seed(42)
idf = pl.DataFrame({'a': range(1_000_000),'b': [np.random.choice(['s','a','h','i','l']) for x in range(1_000_000)]})
idf.write_parquet(str(BASE_DIR/"my_model"/"datasets"/"custom_df.parquet"))

In [93]:
my_df = load_dataset("parquet",data_files=str(BASE_DIR/"my_model"/"datasets"/"custom_df.parquet"))

Generating train split: 1000000 examples [00:00, 54054488.75 examples/s]


In [97]:
## The function to compute the square of a column. The output has to be formatted as a dict
def compute_square(row)->dict:
    return {'c': row['a']**2}

my_df = my_df.map(compute_square, num_proc=16)

Map (num_proc=16): 100%|██████████| 1000000/1000000 [00:01<00:00, 893598.54 examples/s]


In [109]:
my_df['train']

Dataset({
    features: ['a', 'b', 'c'],
    num_rows: 1000000
})

In [118]:
## dataset sharding for parallel processing
print(my_df.shape)

my_df['train'].shard(num_shards=6, index = 0)


{'train': (1000000, 3)}


Dataset({
    features: ['a', 'b', 'c'],
    num_rows: 166667
})